# Intrinsieke camera kalibratie


> In dit script gaan we de camera kalibreren. Dit wil zeggen dat we opzoek zijn naar de intrinsieke camera matrix. Deze matrix zet metrische coördinaten (X, Y, Z) gedefiniëerd in het cameraframe {C} om in pixelcoördinaten (u, v). We kunnen dit schrijven in matrix notatie:

$$\begin{bmatrix} u'\\ v' \\ w' \end{bmatrix} = \begin{bmatrix} \frac{f_{x}}{\rho_{w}} & 0 & u_{0} \\ 0 & \frac{f_{y}}{\rho_{h}} & v_{0} \\ 0 & 0 & 1 \end{bmatrix}*\begin{bmatrix} X\\ Y \\ Z \end{bmatrix}$$ 

> Waarbij w' een schaal factor is en de werkelijke pixelcoördinaten kunnen worden voorgesteld door: $u = \frac{u'}{w'}$ en $v = \frac{v'}{w'}$. Als we dit stelsel uitschrijven dan zien we inderdaad dat :

$$u'=\frac{f_{x}*X}{\rho_{w}} + u_{0}*Z$$

$$v'=\frac{f_{y}*Y}{\rho_{h}} + v_{0}*Z$$

> En dus:

$$u=\frac{f_{x}*X}{\rho_{w}*Z} + u_{0}$$

$$v=\frac{f_{y}*Y}{\rho_{h}*Z} + v_{0}$$

> We zien dus inderdaad dat als we het metrische coördinaat van een punt beschreven in cameraframe (X, Y, Z) kennen, dat we het overeenkomstige pixelcoördinaat (u,v) kunnen bepalen. Let erop dat we hier een dimensie verliezen door om te zetten van 3D naar 2D. We kunnen uit één beeld geen diepte bepalen. Om de ongekende parameters van de intrinsieke camera matrix te kunnen bepalen hebben we dus een set van punten nodig waarvan we de metrische coördinaten kennen alsook de overeenkomstige pixelcoördinaten. Als we zo genoeg bekenden kunnen genereren kunnen we deze gebruiken om uit het stelsel de onbekenden $(f_{x}, f_{y}, \rho_{w}, \rho_{h}, u_{0}, v_{0})$ op te lossen. Om dit te doen maken we gebruik van een dambordpatroon waar we de exacte afmetingen van kennen. 

# 1. Importeer biliotheken


> Deze bibliotheken zijn nodig voor het volbrengen van dit script. In de CameraCalibration bibliotheek (die je vindt in de Classes folder op je Google Drive) vind je de code achter de commando's die je hier zal gebruiken. Deze cel zal een error geven als je niet met je Google Drive bent verbonden. Zorg er dus voor dat je links in de bestanden een folder 'drive' ziet staan naast de default 'sample_data'-folder. Indien niet, verbind met de drive door links op het mapje te klikken en vervolgens op het Google Drive symbool te klikken. Druk vervolgens nog eens op 'Runtime', 'Runtime opnieuw starten', om te refreshen. 




In [ ]:
import sys
sys.path.append('/content/drive/My Drive/object_pose_estimation_online')
from Classes.CameraCalibration import *
from google.colab.patches import cv2_imshow

# 2. Importeer alle beelden voor camera kalibratie

> Voor de camera kalibratie hebben we beelden nodig van een kalibratietool, we gebruiken een dambordpatroon van 7x7 (hiermee wordt bedoeld dat er 7x7 zwart-zwart overgangen zijn, geen hokjes). Alle afbeeldingen bevinden zich onder de map /content/drive/My Drive/object_pose_estimation_online/data/camera_calibration_images. Met de asteriks hieronder in de padnaam wordt aangegeven dat uit deze map alle afbeelingen startende met "CC_image_original_" bedoeld worden. De methode "get_image_path_names" geeft alle aparte bestandsnamen terug en slaat deze op in "images_path_names". Als je dit uitprint kan je zien dat dit een array is met alle 10 de padnamen naar elke afbeelding. 



In [ ]:
# Get all image's path names
camera_calibration_images_file = '/content/drive/My Drive/object_pose_estimation_online/data/camera_calibration_images/CC_image_original_*.jpg'
images_path_names = get_image_path_names(camera_calibration_images_file)
print(images_path_names)



> Als we een afbeelding willen weergeven kunnen we daar de onderstaande code voor gebruiken. Deze neemt de 1e afbeelding (Python indexeert vanaf 0 en doet dit met vierkante haakjes). We lezen het beeld uit met "read_image" en plotten deze met "cv2_imshow()". 

In [ ]:
# Preview image
image_path = images_path_names[0]
image = read_image(image_path)
cv2_imshow(image)



> Op deze afbeelding zien we dus ons dambordpatroon. Op dit dambordpatroon definiëren we een assenstelsel met x-, y-, en z-as. We definiëren voor elk van de hoekpunten (zwart-zwart overgangen) een metrisch coördinaat. De vakjes van het patroon zijn exact 0.02m breed. We kunnen nu bijvoorbeeld het hoekpunt rechts boven (omcirkeld) definiëren als de oorsprong met metrische coördinaten (X, Y, Z) = (0, 0, 0), het eerste coördinaat langs de x-as als (X, Y, Z) = (0.02, 0, 0), en het eerste coördinaat langs de y-as als (X, Y, Z) = (0, 0.02, 0). Zo kunnen we voor elk van de hoekpunten een metrisch coördinaat (X, Y, Z) definiëren. We kiezen hier z altijd 0. Als we nu met een algoritme in deze foto de hoeken detecteren in pixel coördinaten, dan kunnen we de gedefiniëerde metrische coördinaten (X, Y, Z) matchen met de gedetecteerde pixelcoördinaten (u, v) en deze 49 (7x7) gematchte paren gebruiken om de onbekenden uit de intrinsieke matrix te bepalen.



# 3. Testen van het corner detection algoritme


> Voor de camera kalibratie dienen we de exacte pixelcoördinaten te kennen van alle 49 hoekpunten in het patroon, er wordt eerst meegegeven dat we naar een 7x7 patroon zoeken. 



In [ ]:
# Define chessboard pattern to search for
pattern = (7, 7)



> We zetten de afbeelding om naar grayscale, dit wordt gedaan om data te reduceren (van 3 RGB arrays naar 1 gray array). 



In [ ]:
# Convert RGB image to grayscale
gray = image_to_grayscale(image)
cv2_imshow(gray)



> De functie "find_corners" geeft ons alle 49 pixel coördinaten (u, v) van elke zwart-zwart overgang in het patroon. Je kan deze uitprinten.



In [ ]:
# Find the chess board corners in the image
corners = find_corners(gray)
print(corners)



> De gedetecteerde hoeken kunnen worden gevisualiseerd op de afbeelding met de functie "drawChessboardCorners" van de cv2 bibliotheek. De kleuren geven aan in welke volgorde de hoeken zijn gedetecteerd en dus ook in welke volgorde deze worden opgeslagen in de array. Het eerste rode hoekpunt wordt als eerste opgeslagen en het laatse violette hoekpunt als laatste. 



In [ ]:
# Show detected corners
imgage = cv2.drawChessboardCorners(image, pattern, corners, True)
cv2_imshow(image)

#4. Voorbereiding van de camera kalibratie met  10 afbeeldingen


> Bij camera kalibratie willen we alle 9 parameters van de intrinsieke camera matrix berekenen. Om voldoende data te hebben gebruiken we 10 afbeeldingen waarbij we telkens de pixelcoördinaten (u, v) van de hoekpunten bepalen en deze opslaan samen met de overeenkomstige metrische coördinaten (X, Y, Z). Deze metrische coördinaten definiëren we manueel. We zeggen bijvoorbeeld dat het eerste hoekpunt dat het algoritme detecteert (rood) overeenkomt met metrisch coördinaat (X, Y, Z) = (0, 0, 0). We definiëren het tweede rode hoekpunt dat het algoritme detecteert als liggende langs de x-as en geven dit dus het metrisch coördinaat (X, Y, Z) = (0.02, 0, 0). De functie "get_object_points" geeft ons al deze 49 manueel gedefiniëerde coördinaten, je kan deze uitprinten. 



In [ ]:
# Get user defined world coordinates
objp = get_object_points()
print(objp)



> In een for-loop zullen alle hoekpunten (corners) van elke afbeelding worden gedetecteerd en gerelateerd aan de metrische coördinaten (objp). Per afbeelding worden deze telkens opgeslagen in 1 grote array dewelke we hier initialiseren alvoren de for-loop te starten. 



In [ ]:
# Initialize empty arrays for world and pixel cooridnates
imgpoints = []
objpoints = []



> In deze for-loop wordt het voorgaande gedaan voor 10 afbeeldingen van het dambordpatroon. Telkens wordt een afbeelding ingelezen, omgezet naar grayscale, hoekpunten gedetecteerd, en worden de corresponderende pixelcoördinaten en metrische coördinaten van elk hoekpunt toegevoegd aan de respectievelijke arras 'imgpoints' en 'objpoints'. Vervolledig de code door gebruik te maken van hetgeen je hiervoor hebt gezien. 

In [ ]:
# Iterate over all images
for image_path in images_path_names:

    # Read image from file
    image = 

    # Convert RGB image to grayscale
    gray = 

    # Find the chess board corners in the image
    corners = 

    # Add object points to objpoints list
    objpoints.append(objp)

    # Add pixel coordinates to imgpoints list
    imgpoints.append(corners)




> Eens de for-loop voorbij is, zouden de arrays een correcte inhoud moeten hebben. Namelijk een array objpoints met 10 maal een 49x3 array, zijnde de metrische coordinaten (X, Y, Z) van de 49 hoekpunten in het patroon (moet (10, 49, 3) weergeven) en een array imgpoint met 10 maal een 49x2 array, zijnde de pixelcoordinaten (u, v) van de 49 hoekpunten in het patroon (moet (10, 49, 1, 2) weergeven). Let op: als je meerdere keren de for-loop uitvoert zonder de arrays 'objpoints' en 'imgpoints' terug te initialiseren, dan blijf je steeds nieuwe data toevoegen aan de arrays en gaan ook hun groottes verschillen van de (10, 49, 3), en (10, 49, 1, 2). Dit is op zich geen probleem, zo krijgt het algoritme voor intrinsieke kalibratie gewoon meer data. 



In [ ]:
# Check dimensions of arrays
print("Dimensions of world coordinate array: " + str(np.shape(objpoints)))
print("Dimensions of pixel coordinate array: " + str(np.shape(imgpoints)))

# 5. Intrinsieke camera kalibratie

> Eens we de twee arrays hebben met de pixelcoördinaten en de overeenkomstige metrische coördinaten, kunnen we deze twee aan een functie "calibrate\_camera" geven die voor ons de intrinsieke matrix zal bepalen. Deze wordt automatisch met de tevens berekende distortie coëfficienten uitgeprint. In de matrix kan je herkennen wat de brandpuntafstand in x- en y-richting is alsook de camera centerpunten $u_{0}$ en $v_{0}$. 



In [ ]:
# Calibrate camera and obtain intrinsic camera matrix using the objpoints list and the imgpoints list
intrinsic_camera_matrix = calibrate_camera(objpoints, imgpoints)



> Tot slot berekenen we nog de 'reprojection error'. Hierbij zal een algoritme de berekende pixelcoördinaten, gebruik makende van je berekende intrinsieke matrix, vergelijken met de pixelcoördinaten uit het corner detectie algoritme. Dit geeft een error die weergeeft hoe goed je kalibratie is (lager dan 0.1 is oké). 



In [ ]:
# Calculate reprojection error
reprojection_error = calculate_reprojection_error(intrinsic_camera_matrix, objpoints, imgpoints)
print("\nTotal reprojection error: %f" % (reprojection_error / float(len(objpoints))))

# 6. Test camera kalibratie



> In wat volgt kunnen we de kalibratie testen. De intrinsieke matrix geeft ons namelijk weer welke pixel coördinaten overeenkomen met objecten gepositioneerd op een bepaalde locatie in het camera frame {C}. We kunnen dus coördinaten van een object die we virtueel plaatsen in het camera frame projecteren in pixel frame (dit is de basis van Augmented Reality). Hierbij laden we eerst terug onze berekende intrinsieke camera matrix.  



In [ ]:
# Load intrinsic camera matrix and distortion coefficients
intrinsic_camera_matrix_file = "/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/intrinsic_camera_properties.npz"
mtx, dist = load_intrinsic_camera_matrix(intrinsic_camera_matrix_file)



> We definiëren twee zaken:


*   De coordinaten van het camera assenstelsel. We definiëren de oorsprong op (X, Y, Z) = (0, 0, 0) in het camera frame, de x-as op (X, Y, Z) = (0.6, 0, 0) , de y-as op (X, Y, Z) = (0, 0.6, 0), en de z-as op (X, Y, Z) = (0, 0, 0.6). 
*   De coördinaten van een virtueel vierkant van 0.04x0.04 meter in het camera frame. Eens op een afstand van 0.5 meter van de camera, 0.6 meter, en 1.7 meter.


> Voor deze beide zaken maken we gebruik van de intrinsieke matrix om te kijken met welke pixels deze overeenkomen. Als we deze pixels berekenen en projecteren op het beeld, dan krijgen we het effect alsof deze zaken zich werkelijk in de ruimte bevinden.







In [ ]:
# Define axis and box coordinates in camera coordinate system
axis = np.float32([[0.06, 0, 0], [0, 0.06, 0], [0, 0, 0.06], [0, 0, 0]])
square1 = np.float32([[0.08, 0.06, 0.5], [0.12, 0.06, 0.5], [0.12, 0.10, 0.5], [0.08, 0.10, 0.5]]).reshape(-1, 3)
square2 = np.float32([[0.08, 0.06, 0.6], [0.12, 0.06, 0.6], [0.12, 0.10, 0.6], [0.08, 0.10, 0.6]]).reshape(-1, 3)
square3 = np.float32([[0.08, 0.06, 1.7], [0.12, 0.06, 1.7], [0.12, 0.10, 1.7], [0.08, 0.10, 1.7]]).reshape(-1, 3)

> Hier projecteren we de gedefiniëerde metrische coördinaten van het assenstelsel (axis) en de vierkanten (squareX) beschreven in het target frame, in pixelcoördinaten en tekenen we deze op de afbeelding. 

In [ ]:
# Project 3D axis points to pixel coordinates
imgpts_axis, _ = cv2.projectPoints(axis, np.identity(3), np.zeros((3, 1)), mtx, dist)

# Project 3D box points to pixel coordinates
imgpts_square1, _ = cv2.projectPoints(square1, np.identity(3), np.zeros((3, 1)), mtx, dist)

# Project 3D box points to pixel coordinates
imgpts_square2, _ = cv2.projectPoints(square2, np.identity(3), np.zeros((3, 1)), mtx, dist)

# Project 3D box points to pixel coordinates
imgpts_square3, _ = cv2.projectPoints(square3, np.identity(3), np.zeros((3, 1)), mtx, dist)

# Draw axis on image
image = draw_axis(image, imgpts_axis)

# Draw square 1 on image
image = draw_square(image, imgpts_square1)

# Draw square 2 on image
image = draw_square(image, imgpts_square2)

# Draw square 3 on image
image = draw_square(image, imgpts_square3)

> Plot hier de afbeelding met de geprojecteerde pixels. 

In [ ]:
# Show result
cv2_imshow(image)



> Het resultaat laat inderdaad zien dat de gedefiniëerde objecten in camera frame coördinaten mooi de juiste pixels triggeren om het effect te creëren dat de objecten effectief in de ruimte aanwezig zijn. Merk op dat het geplotte frame het camera frame {C} voorstelt. De intrinsieke matrix projecteert dus metrische punten in dit coördinatenframe in pixelcoördinaten. De z-as is hier telkens weg van de camera gericht. De oorsprong bevindt zich in het camera centerpunt. Merk ook op dat hoe verder het vierkant van de camera staat (groter z-coördinaat), hoe kleiner dit vierkant wordt en hoe meer dit naar het centrum beweegt, ookal blijven de x- en y- coördinaten in camera frame gelijk. Motiveer dit aan de hand van de vergelijkingen voor het berekenen van de u- en v coördinaat. Je kan zelf wat spelen door de coördinaten van het vierkant te veranderen. 





> Je kan ook zelf kijken welke pixels worden getriggerd door punten in metrische coördinaten, gedefiniëerd in het cameraframe te projecteren in pixel frame.



In [ ]:
# Define point
point = np.float32([0, 0, 0])

# Project 3D point to pixel coordinates
imgpts_point, _ = cv2.projectPoints(point, np.identity(3), np.zeros((3, 1)), mtx, dist)
print(imgpts_point)


*   *VRAAG 1: Wat is het pixelcoördinaat dat overeenkomt met het 3e gedetecteerde hoekpunt in het dambordpatroon?*
*   *VRAAG 2: Hoeveel bedraagt de brandpuntafstand in pixelcoördinaten alsook het camera centerpunt in de x-richting van de gebruikte camera?*
*   *VRAAG 3: Welk pixelcoördinaat wordt er getriggerd als een punt op coördinaat (0.2, 0.2, 0.5) meter in het camera frame wordt geplaatst?*




